In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.datasets import mnist
from keras import Input
import numpy as np

2023-11-21 22:16:32.905042: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 22:16:32.937407: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 22:16:32.938175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 22:16:33.769264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
y_train.shape

(60000,)

In [5]:
x_test.shape

(10000, 28, 28)

In [6]:
y_test.shape

(10000,)

In [7]:
x_train = x_train/255.
x_test = x_test/255.

In [8]:
x_train = x_train.reshape(60000,28*28)
x_test = x_test.reshape(10000,28*28)

In [9]:
model = Sequential([
    Input(shape=(784,)),
    Dense(784,  activation='relu'),
    Dropout(0.2),
    Dense(500, activation='relu'),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')    
])

2023-11-21 22:16:35.550198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 22:16:35.550807: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 784)               615440    
                                                                 
 dropout (Dropout)           (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               392500    
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               50100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                1

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train, epochs=5, validation_split=0.2)

Epoch 1/5


2023-11-21 22:16:38.419994: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 150528000 exceeds 10% of free system memory.


1500/1500 [==============================] - 11s 7ms/step - loss: 0.2671 - accuracy: 0.9203 - val_loss: 0.1099 - val_accuracy: 0.9678
Epoch 2/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.1265 - accuracy: 0.9626 - val_loss: 0.0977 - val_accuracy: 0.9705
Epoch 3/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0978 - accuracy: 0.9713 - val_loss: 0.0888 - val_accuracy: 0.9750
Epoch 4/5
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0797 - accuracy: 0.9767 - val_loss: 0.1032 - val_accuracy: 0.9722
Epoch 5/5
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0697 - accuracy: 0.9789 - val_loss: 0.0849 - val_accuracy: 0.9780


In [61]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 1ms/step - loss: 0.1053 - accuracy: 0.9726
Test accuracy: 0.972599983215332


In [ ]:
model.save('tfmodel.h5')

In [65]:
import onnx
import tf2onnx
import onnxruntime as rt

In [ ]:
onnx_model, _ = tf2onnx.convert.from_keras(model)

In [66]:
onnx.save(onnx_model, 'onnxmodel.onnx')

In [68]:
session = rt.InferenceSession('onnxmodel.onnx')

In [69]:
input_name = session.get_inputs()[0].name
print(input_name)
output_name = session.get_outputs()[0].name
print(output_name)

input_7
dense_35


In [100]:
session.get_inputs()[0]

'input_7'

In [70]:
session.get_inputs()[0].shape

['unk__6', 784]

In [73]:
session.get_inputs()[0].type

'tensor(float)'

In [87]:
test = x_test[0]

In [101]:
test = test.reshape(28*28).astype(np.float32)
test = np.expand_dims(test, axis=0)

In [102]:
test.shape

(1, 784)

In [103]:
test.dtype

dtype('float32')

In [104]:
res = session.run([output_name], {input_name : test})

In [105]:
np.argmax(res)

7

In [106]:
y_test[0]

7

In [107]:
def onnxPred(input):
    test = input.reshape(28*28).astype(np.float32)
    test = np.expand_dims(test, axis=0)
    res = session.run([output_name], {input_name : test})
    return np.argmax(res)

In [108]:
onnxPred(x_test[1])

2